In [1]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:03<00:00, 11.6MB/s]

Extracting files...


Path to dataset files: C:\Users\D.Ivanovas\.cache\kagglehub\datasets\clmentbisaillon\fake-and-real-news-dataset\versions\1


In [14]:
import pandas as pd
import os 
fake_df = pd.read_csv(os.path.join(path, 'fake.csv'))
true_df = pd.read_csv(os.path.join(path, 'true.csv'))

fake_df["label"]=0
true_df["label"]=1



In [ ]:
joined_df = pd.concat([fake_df, true_df], axis=0, ignore_index=True) # sujungiam sulablintus datasetus
joined_df = joined_df.sample(frac=1).reset_index(drop=True) # sushufflinam dataseta
joined_df

,title,text,subject,date,label
0,Here’s How The Bundy Terrorists Left The Malh...,"On March 23, KOIN in Burns, Oregon, got a beh...",News,"March 24, 2016",0
1,Obama To U.S.Military On Putin: “he did not ta...,Sarah Palin can hear Putin s knees knocking fr...,politics,"Sep 13, 2015",0
2,HOUSE TAKES Measures To Impeach IRS Chief Afte...,Speaker Paul Ryan said Wednesday the House wil...,Government News,"Sep 8, 2016",0
3,"WATCH: LAST WEEK Giants’ Odell Beckham Jr., “P...","Last week, after the New York Giants player Od...",left-news,"Oct 8, 2017",0
4,Trump Makes New Racist Attack On Judge Exposi...,Donald Trump is very upset at the judge overse...,News,"June 3, 2016",0
...,...,...,...,...,...
44893,MURDERERS AND RAPISTS FOR HILLARY: VA Governor...,The good news for Hillary is that she should b...,left-news,"Apr 23, 2016",0
44894,"Hoyer, asked on German finmin post, says 'extr...",ATHENS (Reuters) - European Investment Bank (E...,worldnews,"September 29, 2017",1
44895,"Liberal, Man-Hating Movement DESTROYS Career O...",Lauren McGaughy of The Dallas Morning News di...,left-news,"Dec 16, 2017",0
44896,EU to review Brexit approach if no deal by Dec...,BRUSSELS (Reuters) - The European Union will h...,worldnews,"October 10, 2017",1


In [3]:
from paths import DATA_DIR
import shutil

shutil.move(str(path), str(DATA_DIR))

'/mnt/c/Users/atrol/PycharmProjects/Fake-News/data/1'